In [ ]:
from datetime import datetime, timezone, timedelta
import numbers



class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError("Timezone name cannot be empty.")
        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("Hour offset must be an integer")
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError("Minute offset must be an integer.")

        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError("Minutes offset must be between -59 and 59 (inclusive).")
        
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError("Offset must be between -12:00 and +14:00.")
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return(isinstance(other, TimeZone) and 
                self.name == other.name and 
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)

    def __repr__(self):
        return (f"TimeZone(name='{self._name}' "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")


tz1 = TimeZone("ABC", -2, -15)

In [2]:
tz1.offset

datetime.timedelta(days=-1, seconds=78300)

In [90]:
class Account:

    _interest_rate = 0.05
    transaction_num = 0
    account_nums = []
    
    def __init__(self, account_num, first_name, last_name, tz_name="Default", tz_offset_hours=0, tz_offset_minutes=0):
        if not isinstance(account_num, numbers.Integral):
            raise ValueError("Account number must be an integer.")
        # global account_nums
        # if account_num in account_nums:
        #     raise ValueError(f"The account with number {account_num} already exists.")
        self._account_num = account_num
        # account_nums.append(account_num)

        if not isinstance(first_name, str) or len(str(first_name).strip()) == 0:
            raise ValueError("Please provide a valid first name.")
        self._first_name = first_name

        if not isinstance(last_name, str)or len(str(last_name).strip()) == 0:
            raise ValueError("Please provide a valid last name.")
        self._last_name = last_name
        
        self._time_zone = TimeZone(tz_name, tz_offset_hours, tz_offset_minutes)

        self._balance = 0

    @property
    def account_num(self):
        return self._account_num
    
    @property
    def first_name(self):
        return self._first_name
    
    @property
    def last_name(self):
        return self._last_name

    @property
    def full_name(self):
        return f"{self._first_name} {self._last_name}"
    
    @property
    def time_zone(self):
        return self._time_zone
    
    @property
    def balance(self):
        return self._balance
    
    @first_name.setter
    def first_name(self, new_first_name):
        if not isinstance(new_first_name, str) or len(str(new_first_name).strip()) == 0:
            raise ValueError("Please provide a valid first name.")
        old_first_name = self.first_name
        self._first_name = new_first_name
        print(f"Successfully changed First Name from: {old_first_name} to {self.first_name}")

    @last_name.setter
    def last_name(self, new_last_name):
        if not isinstance(new_last_name, str) or len(str(new_last_name).strip()) == 0:
            raise ValueError("Please provide a valid last name.")
        old_last_name = self.last_name
        self._last_name = new_last_name
        print(f"Successfully changed Last Name from: {old_last_name} to {self.last_name}")
    
    @time_zone.setter
    def time_zone(self, param_tuple):
        tz_name, tz_offset_hours, tz_offset_minutes = param_tuple
        self._time_zone = TimeZone(tz_name, tz_offset_hours, tz_offset_minutes)
        print(f"Successfully changed the timezone to the following: {self.time_zone}")


    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, new_interest_rate):
        cls._interest_rate = new_interest_rate
        print(f"Successfully updated interest rate to {new_interest_rate}")
    

    def generate_transaction(self):
        transaction_datetime = datetime.now(timezone.utc)
        cur_transaction = Account.transaction_num
        Account.transaction_num += 1
        return transaction_datetime, cur_transaction


    def return_transaction_info(self, transaction_datetime, cur_transaction, transact_status):
        return (f"{transact_status}-"
                f"{self.account_num}-"
                f"{transaction_datetime.strftime('%Y')}"
                f"{transaction_datetime.strftime('%m')}"
                f"{transaction_datetime.strftime('%d')}"
                f"{transaction_datetime.strftime('%H')}"
                f"{transaction_datetime.strftime('%M')}"
                f"{transaction_datetime.strftime('%S')}-"
                f"{cur_transaction}")


    def deposit(self, deposit_value):
        transaction_datetime, cur_transaction = self.generate_transaction()
        if not isinstance(deposit_value, (int, float)):
            try:
                raise ValueError("Please provide a valid amount.")
            except ValueError as e:
                print(f"{e}")
                return self.return_transaction_info(transaction_datetime, cur_transaction, "X")
        
        self._balance = self._balance + deposit_value
        print(f"Successfully deposited ${deposit_value}")
        return self.return_transaction_info(transaction_datetime, cur_transaction, "D")


    def withdraw(self, withdrawal_value):
        transaction_datetime, cur_transaction = self.generate_transaction()
        try:
            if not isinstance(withdrawal_value, (int, float)):
                raise ValueError("Please provide a valid amount.")
            if self.balance - withdrawal_value < 0: 
                raise ValueError(f"Not enough money on your balance to withdraw {withdrawal_value}\nCurrent balance: {self.balance}")
            
        except ValueError as e:
            print(f"{e}")
            return self.return_transaction_info(transaction_datetime, cur_transaction, "X")
        self._balance = self._balance - withdrawal_value
        print(f"Successfully withdrawn {withdrawal_value}")
        return self.return_transaction_info(transaction_datetime, cur_transaction, "W")
    

    def add_interest_rate(self):
        interest_amount = self.balance * Account.interest_rate()
        print(f"The amount of interest is: {interest_amount}")
        transaction_datetime, cur_transaction = self.generate_transaction()
        self.deposit(interest_amount)
        return self.return_transaction_info(transaction_datetime, cur_transaction, "I")
    

    @classmethod
    def decode_conf_number(cls, conf_number, tz_info=None):
        transaction_code, account_number, datetime_string, transaction_number = conf_number.split("-")
        year = int(datetime_string[:4])
        month = int(datetime_string[4:6])
        day = int(datetime_string[6:8])
        hour = int(datetime_string[8:10])
        minute = int(datetime_string[10:12])
        second = int(datetime_string[12:14])

        transaction_datetime_utc = datetime(year=year,
                                            month=month,
                                            day=day,
                                            hour=hour,
                                            minute=minute,
                                            second=second,
                                            tzinfo=timezone.utc)
        
        if tz_info is not None:
            transaction_datetime = transaction_datetime_utc.astimezone(timezone(tz_info.offset, tz_info.name))
        else:
            transaction_datetime = transaction_datetime_utc

        transaction_datetime_utc_string = transaction_datetime_utc.strftime("%Y-%m-%d %H:%M:%S")
        transaction_datetime_string = transaction_datetime.strftime("%Y-%m-%d %H:%M:%S") + f" ({tz_info.name})"
        
        return {"transaction_code": transaction_code,
                "account_number": account_number,
                "transaction_datetime":  transaction_datetime_string,
                "transaction_datetime_utc": transaction_datetime_utc_string,
                "transaction_number": transaction_number}
        

    def __repr__(self):
        return (f"Account Number: {self.account_num}\n"
                f"First Name: {self.first_name}\n"
                f"Last Name: {self.last_name}\n"
                f"Timezone: {self.time_zone}\n"
                f"Balance: ${self.balance}")


my_acc = Account(account_num=123, 
                 first_name="Slava", 
                 last_name="Calestru", 
                 tz_name="ABC",
                 tz_offset_hours=1,
                 tz_offset_minutes=10)


my_acc

Account Number: 123
First Name: Slava
Last Name: Calestru
Timezone: TimeZone(name='ABC' offset_hours=1, offset_minutes=10)
Balance: $0

In [93]:
my_acc.deposit(100)

Successfully deposited $100


'D-123-20250303120951-2'

In [89]:
Account.decode_conf_number('D-123-20250303111534-6', my_acc.time_zone)

{'transaction_code': 'D',
 'account_number': '123',
 'transaction_datetime': '2025-03-03 12:25:34 (ABC)',
 'transaction_datetime_utc': '2025-03-03 11:15:34',
 'transaction_number': '6'}

In [86]:
Account.set_interest_rate(0.1)

Successfully updated interest rate to 0.1


In [87]:
my_acc.get_interest_rate()

0.1

In [68]:
my_acc.add_interest_rate()

The amount of interest is: 0.0
Successfully deposited $0.0


'I-123-20250303113406-28'

In [50]:
my_acc

Account Number: 123
First Name: Slava
Last Name: Calestru
Timezone: TimeZone(name='ABC' offset_hours=1, offset_minutes=0)
Balance: $107.625

In [51]:
my_acc.withdraw(1000)

Not enough money on your balance to withdraw 1000
Current balance: 107.625


'X-123-20250303112809-26'

{'transaction_status': 'D',
 'account_number': '123',
 'transaction_datetime': datetime.datetime(2025, 3, 3, 12, 15, 34, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'ABC')),
 'transaction_datetime_utc': datetime.datetime(2025, 3, 3, 11, 15, 34, tzinfo=datetime.timezone.utc),
 'transaction_number': '6'}

In [79]:
Account.interest_rate()

0.05